## **Module 1**

In [1]:
import pandas as pd
import os

In [ ]:

# Create empty DataFrame
columns = ["Course", "Grade"]
df = pd.DataFrame(columns=columns)
# Open file in write mode
with open("grades.txt", "w", encoding="utf-8") as file:
    file.write("Grades Summary\n==============\n")

    while True:
        name = input("Enter course name (or 'done' to finish): ").strip().title()
        if name.lower() == "done":
            break

        try:
            grade = float(input(f"Enter {name}'s grade: "))
            # Add to DataFrame
            df.loc[len(df)] = [name, grade]
            # Write to file immediately
            file.write(f"{name}: {grade}\n")
        except ValueError:
            print("Please enter a valid number for grade.")


In [5]:
# Calculate summary
average = df["Grade"].mean()
best_subject = df.loc[df["Grade"].idxmax(), "Course"]
best_grade = df["Grade"].max()

weakest_subject = df.loc[df["Grade"].idxmin(), "Course"]
weakest_grade = df["Grade"].min()
print(f"\nAverage grade: {average:.2f}")
print(f"Best subject: {best_subject} ({best_grade})")
print(f"Weakest subject: {weakest_subject} ({weakest_grade})")


# Append summary
with open("grades.txt", "a", encoding="utf-8") as file:
    file.write("\nResults\n")
    file.write(f"Average: {average:.2f}\n")
    file.write(f"Best subject: {best_subject} ({best_grade})\n")
    file.write(f"Weakest subject: {weakest_subject} ({weakest_grade})\n")

print("\nGrades DataFrame:\n", df)
print("\nSaved to grades.txt ")



Average grade: 79.60
Best subject: Computer Science (100.0)
Weakest subject: History (40.0)

Grades DataFrame:
              Course  Grade
0              Math   99.0
1  Computer Science  100.0
2           History   40.0
3           English   70.0
4       Programming   89.0

Saved to grades.txt 


## **Module_2**

In [8]:
# Study Assistant (Rule-Based) — Pandas version
# Reads grades from grades.txt, builds a DataFrame, and chats with the user.
# Commands: help, summary, advice, best, weak, tips, plan, exit




def load_df_from_txt(path: str) -> pd.DataFrame:
    """Parse 'Course: grade' lines from grades.txt into a DataFrame [Course, Grade]."""
    if not os.path.exists(path):
        raise FileNotFoundError("grades.txt not found. Please run Module 1 first.")

    rows: list[list] = []
    with open(path, "r", encoding="utf-8") as f:
        for raw in f:
            line = raw.strip()
            # Skip empty and summary lines
            if not line or line.startswith(("Grades", "Results", "Average", "Best", "Weakest", "=")):
                continue
            # Accept valid lines that contain a colon
            if ":" in line:
                left, right = line.split(":", 1)
                name = " ".join(left.split()).title()
                try:
                    grade = float(right.strip())
                except ValueError:
                    continue
                rows.append([name, grade])

    if not rows:
        raise ValueError("No valid 'Course: Grade' lines found in grades.txt.")
    return pd.DataFrame(rows, columns=["Course", "Grade"])

def compute_summary(df: pd.DataFrame) -> dict:
    """Compute the average, best, and weakest subjects."""
    avg = df["Grade"].mean()
    best_idx = df["Grade"].idxmax()
    weak_idx = df["Grade"].idxmin()
    return {
        "average": avg,
        "best": df.loc[best_idx],
        "weak": df.loc[weak_idx],
        "count": len(df),
    }

def motivation(avg: float) -> str:
    """Return a motivational message based on the average."""
    if avg >= 85:
        return " Excellent job! Keep up the great work!"
    if 60 <= avg < 85:
        return " Good effort! Keep reviewing your weak subjects."
    return " Let's plan to improve next week. You can do it!"

def weekly_plan(weak_subject: str) -> str:
    """Return a 7-day plan focused on the weakest subject."""
    return (
        f"\n 7-Day Study Plan (Focus: {weak_subject})\n"
        f"Day 1: Revise fundamentals (30–45 minutes)\n"
        f"Day 2: Practice 10 problems in {weak_subject}\n"
        f"Day 3: Review mistakes and summarize key points\n"
        f"Day 4: Combine {weak_subject} with another subject\n"
        f"Day 5: Timed quiz (15–20 minutes)\n"
        f"Day 6: Teach the topic aloud (explain to yourself)\n"
        f"Day 7: Light review + rest \n"
    )

def show_help():
    """Display available chatbot commands."""
    print(
        "\nAvailable Commands:\n"
        "- summary : Show all subjects, grades, and average\n"
        "- advice  : Motivational message based on your average\n"
        "- best    : Show your best subject\n"
        "- weak    : Show your weakest subject\n"
        "- tips    : Display general study tips\n"
        "- plan    : Generate a 7-day plan for your weak subject\n"
        "- help    : Show this help message\n"
        "- exit    : Quit the chatbot\n"
    )

def detect_intent(text: str) -> str:
    """Simple keyword-based intent detection."""
    t = text.strip().lower()
    if t in ("help", "?"): return "help"
    if t in ("summary", "grades"): return "summary"
    if t in ("advice", "motivation"): return "advice"
    if t in ("best", "top"): return "best"
    if t in ("weak", "lowest", "worst"): return "weak"
    if t in ("tips", "study tips"): return "tips"
    if t in ("plan", "schedule", "study plan"): return "plan"
    if t in ("exit", "quit"): return "exit"

    # fuzzy matching
    if "summary" in t: return "summary"
    if "advice" in t or "motivation" in t: return "advice"
    if "best" in t: return "best"
    if "weak" in t or "worst" in t: return "weak"
    if "tips" in t: return "tips"
    if "plan" in t: return "plan"
    if "exit" in t or "quit" in t: return "exit"
    return "unknown"

def print_summary(df: pd.DataFrame, s: dict):
    """Display grades DataFrame and summary statistics."""
    print("\n Your Grades:")
    print(df.to_string(index=False))
    print("\n Summary:")
    print(f"Average Grade   : {s['average']:.2f}")
    print(f"Best Subject    : {s['best']['Course']} ({s['best']['Grade']:.2f})")
    print(f"Weakest Subject : {s['weak']['Course']} ({s['weak']['Grade']:.2f})")

def main():
    """Main chatbot loop."""
    try:
        df = load_df_from_txt("grades.txt")
    except Exception as e:
        print(f"[Error] {e}")
        return

    summary = compute_summary(df)
    #print_summary(df, summary)

    print("\n Study Assistant Chatbot is ready! Type 'help' to see all commands.")
    while True:
        user = input("\nYou: ")
        cmd = detect_intent(user)

        if cmd == "help":
            show_help()
        elif cmd == "summary":
            print_summary(df, summary)
        elif cmd == "advice":
            print("\n Advice:")
            print(motivation(summary["average"]))
        elif cmd == "best":
            print(f"\n Best Subject: {summary['best']['Course']} ({summary['best']['Grade']:.2f})")
        elif cmd == "weak":
            print(f"\n Weakest Subject: {summary['weak']['Course']} ({summary['weak']['Grade']:.2f})")
        elif cmd == "tips":
            print(
                "\n Study Tips:\n"
                "- Use the Pomodoro method (25 min focus + 5 min break)\n"
                "- Review topics after 1, 3, and 7 days\n"
                "- Take short quizzes and analyze your mistakes\n"
                "- Sleep well and stay hydrated "
            )
        elif cmd == "plan":
            print(weekly_plan(summary["weak"]["Course"]))
        elif cmd == "exit":
            print("\n Goodbye! Keep learning and improving every day.")
            break
        else:
            print(" I didn’t understand that. Type 'help' to see available commands.")

if __name__ == "__main__":
    main()



 Study Assistant Chatbot is ready! Type 'help' to see all commands.

You: help

Available Commands:
- summary : Show all subjects, grades, and average
- advice  : Motivational message based on your average
- best    : Show your best subject
- weak    : Show your weakest subject
- tips    : Display general study tips
- plan    : Generate a 7-day plan for your weak subject
- help    : Show this help message
- exit    : Quit the chatbot


You: summary

 Your Grades:
          Course  Grade
            Math   99.0
Computer Science  100.0
         History   40.0
         English   70.0
     Programming   89.0

 Summary:
Average Grade   : 79.60
Best Subject    : Computer Science (100.00)
Weakest Subject : History (40.00)

You: advice

 Advice:
 Good effort! Keep reviewing your weak subjects.

You: advice

 Advice:
 Good effort! Keep reviewing your weak subjects.

You: best

 Best Subject: Computer Science (100.00)

You: advice

 Advice:
 Good effort! Keep reviewing your weak subjects.

You:

## **Module_3**

In [10]:
# Module 3 — Study Time Planner
# Reads subjects from grades.txt, collects weekly study hours per subject,
# computes totals/averages/most/least studied, and saves a report to study_hours.txt.




# ---------- Helpers: load subjects from Module 1 output ----------

def load_subjects_from_grades(path: str) -> list[str]:
    """Extract subject names from grades.txt (lines like 'Subject: 98.0')."""
    if not os.path.exists(path):
        raise FileNotFoundError("grades.txt not found. Please run Module 1 first.")

    subjects: list[str] = []
    with open(path, "r", encoding="utf-8") as f:
        for raw in f:
            line = raw.strip()
            # Skip headers/summary lines
            if not line or line.startswith(("Grades", "Results", "Average", "Best", "Weakest", "=")):
                continue
            if ":" in line:
                left, right = line.split(":", 1)
                name = " ".join(left.split()).title()
                # accept only numeric grade lines
                try:
                    float(right.strip())
                except ValueError:
                    continue
                subjects.append(name)
    if not subjects:
        raise ValueError("No subjects found in grades.txt (expected 'Name: grade').")
    # De-duplicate while preserving order
    seen = set()
    unique_subjects = []
    for s in subjects:
        if s not in seen:
            seen.add(s)
            unique_subjects.append(s)
    return unique_subjects

# ---------- Collect weekly hours from the user ----------

def collect_weekly_hours(subjects: list[str]) -> pd.DataFrame:
    """Prompt the user for hours studied this week for each subject."""
    rows = []
    print("\nEnter the STUDY HOURS for THIS WEEK (e.g., 0, 1.5, 2.25).")
    for subj in subjects:
        while True:
            val = input(f"- {subj} hours: ").strip()
            try:
                hours = float(val)
                if hours < 0:
                    print("  Hours cannot be negative. Try again.")
                    continue
                rows.append([subj, hours])
                break
            except ValueError:
                print("  Please enter a valid number (e.g., 0, 1.5).")
    return pd.DataFrame(rows, columns=["Subject", "Hours"])

# ---------- Analytics ----------

def summarize_hours(df: pd.DataFrame) -> dict:
    """Return total hours, average per subject, most/least studied subjects."""
    total = df["Hours"].sum()
    avg_per_subject = df["Hours"].mean()
    most_idx = df["Hours"].idxmax()
    least_idx = df["Hours"].idxmin()
    return {
        "total_hours": total,
        "avg_per_subject": avg_per_subject,
        "most": df.loc[most_idx],   # row with Subject + Hours
        "least": df.loc[least_idx], # row with Subject + Hours
        "count": len(df),
    }

def quick_feedback(summary: dict) -> str:
    """A small feedback string based on balance and total effort."""
    msgs = []
    # Overall effort
    if summary["total_hours"] >= 20:
        msgs.append("Great overall effort this week! ")
    elif summary["total_hours"] >= 10:
        msgs.append("Good progress—keep the momentum going! ")
    else:
        msgs.append("Let's try to increase study time next week. You can do it! 📘")
    # Balance check
    diff = summary["most"]["Hours"] - summary["least"]["Hours"]
    if diff >= 5:
        msgs.append(f"Balance tip: you spent much more time on {summary['most']['Subject']} than {summary['least']['Subject']}. Try to spread attention more evenly.")
    else:
        msgs.append("Nice balance across subjects this week ")
    return " ".join(msgs)

# ---------- Save report ----------

def save_hours_report(path: str, df: pd.DataFrame, summary: dict) -> None:
    """Write a clean text report to study_hours.txt."""
    with open(path, "w", encoding="utf-8") as file:
        file.write("Study Hours — Weekly Report\n")
        file.write("===========================\n\n")
        for _, row in df.iterrows():
            file.write(f"- {row['Subject']}: {row['Hours']:.2f} h\n")
        file.write("\nResults\n-------\n")
        file.write(f"Total study hours      : {summary['total_hours']:.2f} h\n")
        file.write(f"Average per subject    : {summary['avg_per_subject']:.2f} h\n")
        file.write(f"Most studied subject   : {summary['most']['Subject']} ({summary['most']['Hours']:.2f} h)\n")
        file.write(f"Least studied subject  : {summary['least']['Subject']} ({summary['least']['Hours']:.2f} h)\n")
        file.write("\nFeedback\n--------\n")
        file.write(quick_feedback(summary) + "\n")

# ---------- Main ----------

def main():
    try:
        subjects = load_subjects_from_grades("grades.txt")
    except Exception as e:
        print(f"[Error] {e}")
        return

    print("\nSubjects loaded from grades.txt:")
    print(", ".join(subjects))

    df = collect_weekly_hours(subjects)
    summary = summarize_hours(df)

    # Show console summary
    print("\nYour Study Hours")
    print(df.to_string(index=False))
    print("\nSummary")
    print("-------")
    print(f"Total study hours     : {summary['total_hours']:.2f} h")
    print(f"Average per subject   : {summary['avg_per_subject']:.2f} h")
    print(f"Most studied          : {summary['most']['Subject']} ({summary['most']['Hours']:.2f} h)")
    print(f"Least studied         : {summary['least']['Subject']} ({summary['least']['Hours']:.2f} h)")
    print("\nFeedback:")
    print(quick_feedback(summary))

    # Save to file
    save_hours_report("study_hours.txt", df, summary)
    print("\n Saved report to study_hours.txt")

if __name__ == "__main__":
    main()



Subjects loaded from grades.txt:
Math, Computer Science, History, English, Programming

Enter the STUDY HOURS for THIS WEEK (e.g., 0, 1.5, 2.25).
- Math hours: 3
- Computer Science hours: 5
- History hours: 6
- English hours: 7
- Programming hours: 8

Your Study Hours
         Subject  Hours
            Math    3.0
Computer Science    5.0
         History    6.0
         English    7.0
     Programming    8.0

Summary
-------
Total study hours     : 29.00 h
Average per subject   : 5.80 h
Most studied          : Programming (8.00 h)
Least studied         : Math (3.00 h)

Feedback:
Great overall effort this week!  Balance tip: you spent much more time on Programming than Math. Try to spread attention more evenly.

 Saved report to study_hours.txt


## **Module_4**

In [13]:
# Module 4 — Study Quiz Challenge
# Conducts a short quiz, scores the result, saves to quiz_scores.txt, and gives feedback.


import datetime

# ---------- Quiz Data ----------

QUESTIONS = [
    {
        "q": "Which function is used to find the maximum value in a list in Python?",
        "options": ["max()", "maximum()", "largest()", "top()"],
        "answer": 1,  # 1-indexed: option 1 = correct
    },
    {
        "q": "What does 'df.head()' do in pandas?",
        "options": [
            "Shows the first few rows of a DataFrame",
            "Deletes the first few rows",
            "Sorts the DataFrame by index",
            "Creates a new column called 'head'"
        ],
        "answer": 1,
    },
    {
        "q": "Which file format is commonly used for storing tabular data?",
        "options": ["TXT", "CSV", "JSON", "HTML"],
        "answer": 2,
    },
    {
        "q": "In machine learning, which step comes first?",
        "options": ["Model Evaluation", "Feature Selection", "Data Collection", "Hyperparameter Tuning"],
        "answer": 3,
    },
    {
        "q": "Which keyword is used to handle exceptions in Python?",
        "options": ["if", "raise", "catch", "try"],
        "answer": 4,
    },
]

# ---------- Helper: Run Quiz ----------

def run_quiz() -> dict:
    """Run a short multiple-choice quiz and return total and correct counts."""
    print("\n Study Quiz Challenge — Let's test your knowledge!\n")
    correct = 0
    for i, q in enumerate(QUESTIONS, 1):
        print(f"Q{i}: {q['q']}")
        for j, opt in enumerate(q["options"], 1):
            print(f"  {j}. {opt}")
        while True:
            ans = input("Your answer (1-4): ").strip()
            if ans in {"1", "2", "3", "4"}:
                if int(ans) == q["answer"]:
                    correct += 1
                break
            else:
                print("  Please enter a number between 1 and 4.")
        print()
    total = len(QUESTIONS)
    score = round((correct / total) * 100, 2)
    print(f" You scored {correct}/{total} ({score}%)")
    return {"total": total, "correct": correct, "score": score}

# ---------- Helper: Save Results ----------

def save_quiz_result(path: str, result: dict):
    """Append quiz result with date to quiz_scores.txt and compute summary."""
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    new_entry = {
        "Date": now,
        "Total": result["total"],
        "Correct": result["correct"],
        "Score": result["score"]
    }

    # Create file if not exists
    if not os.path.exists(path):
        df = pd.DataFrame([new_entry])
    else:
        df = pd.read_csv(path)
        df = pd.concat([df, pd.DataFrame([new_entry])], ignore_index=True)

    df.to_csv(path, index=False)

    best = df["Score"].max()
    last = df.iloc[-1]["Score"]

    print("\n Quiz Summary")
    print("-------------------")
    print(f"Best Score : {best}%")
    print(f"Last Score : {last}%")
    diff = last - df["Score"].iloc[-2] if len(df) > 1 else 0
    if diff > 0:
        print(f"Improvement: +{diff:.2f}% ")
    elif diff < 0:
        print(f"Decline: {diff:.2f}% — keep practicing!")
    else:
        print("No change since last attempt.")

    # Feedback
    if last >= 85:
        print(" Excellent performance! Keep mastering advanced topics.")
    elif last >= 60:
        print(" Good job! Review your weak areas to improve further.")
    else:
        print("Don't worry, every expert was once a beginner. Try again tomorrow!")

# ---------- Main ----------

def main():
    print("=== Module 4 — Study Quiz Challenge ===")
    result = run_quiz()
    save_quiz_result("quiz_scores.txt", result)

if __name__ == "__main__":
    main()


=== Module 4 — Study Quiz Challenge ===

 Study Quiz Challenge — Let's test your knowledge!

Q1: Which function is used to find the maximum value in a list in Python?
  1. max()
  2. maximum()
  3. largest()
  4. top()
Your answer (1-4): 1

Q2: What does 'df.head()' do in pandas?
  1. Shows the first few rows of a DataFrame
  2. Deletes the first few rows
  3. Sorts the DataFrame by index
  4. Creates a new column called 'head'
Your answer (1-4): 2

Q3: Which file format is commonly used for storing tabular data?
  1. TXT
  2. CSV
  3. JSON
  4. HTML
Your answer (1-4): 3

Q4: In machine learning, which step comes first?
  1. Model Evaluation
  2. Feature Selection
  3. Data Collection
  4. Hyperparameter Tuning
Your answer (1-4): 3

Q5: Which keyword is used to handle exceptions in Python?
  1. if
  2. raise
  3. catch
  4. try
Your answer (1-4): 4

 You scored 3/5 (60.0%)

 Quiz Summary
-------------------
Best Score : 100.0%
Last Score : 60.0%
Decline: -40.00% — keep practicing!
 Goo

# **Challenge**

In [16]:
grades = {"Omar": 88, "Sara": 91, "Laila": 77, "Ahmed": 91}

# Find the highest grade
max_grade = max(grades.values())

# Find all students who have that grade
top_students = [name for name, grade in grades.items() if grade == max_grade]

# Print the result
print("Top performers:", ", ".join(top_students))


Top performers: Sara, Ahmed


## **Bouns**

In [18]:
study_hours = {"Math": 3, "Python": 5, "AI": 4}

total = 0
for h in study_hours.values():
    total += h

print("Total study hours:", total)


Total study hours: 12
